# Deep Learning for lung diseases classification with Gradio (Graphical interface)
Óscar Poblete Sáenz <br>
Course: Introduction to Artificial Intelligence <br>
Teacher: Elizabeth Guevara Martinez<br>
Universidad Anáhuac <br> <br>
The goal is to develop a model to detect and classify lung diseases from **X-rays** images of the chest.
There are 133 images that belong to **4 classes**:


*   Healthy
*   Covid-19
*   Bacterial pneumonia
*   Viral pneumonia <br><br>

Images from the Université de Montréal project:
https://github.com/ieee8023/covid-chestxray-dataset <br>
Based on the work carried out by a group from Telefónica Digital España S.L.U.
https://colab.research.google.com/drive/1D31N0nvx2VXMPNdefYjW4GRha1MrxlJr?usp=drive_open#scrollTo=lIsvnVsLTm_w

In [1]:
pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.8 MB 28.8 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 56 kB 6.1 MB/s 
     |████████████████████████████████| 278 kB 69.9 MB/s 
     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 106 kB 89.5 MB/s 
     |████████████████████████████████| 2.3 MB 68.2 MB/s 
     |████████████████████████████████| 64 kB 3.3 MB/s 
     |████████████████████████████████| 80 kB 10.5 MB/s 
     |████████████████████████████████| 69 kB 9.0 MB/s 
     |████████████████████████████████| 58 kB 7.2 MB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4711 sha256=0e71a3d0818dd0011f5d260ddf731eba05312790edfb481d849b3becb21bd513
  Stored in dire

In [2]:
# Import libraries
import tensorflow.keras # Tensorflow
import numpy as np # Linear algebra
import cv2 # cv2 is the import name of the module for opencv-python. It allows to perform image processing and artificial vision tasks.
import gradio as gr # Gradio
from skimage import io # Reads the image, converts it from JPG to a NumPy array

In [ ]:
# Load obtained model in Teachable Machine
model = tensorflow.keras.models.load_model('/content/keras_model.h5')

In [4]:
# CNN architecture
# In teachable machine the first layer is convolution and the last layer is classification
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 1280)              410208    
                                                                 
 sequential_3 (Sequential)   (None, 4)                 128500    
                                                                 
Total params: 538,708
Trainable params: 524,628
Non-trainable params: 14,080
_________________________________________________________________


In [5]:
# First layer
model.get_layer(name='sequential_1').summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model1 (Functional)         (None, 7, 7, 1280)        410208    
                                                                 
 global_average_pooling2d_Gl  (None, 1280)             0         
 obalAveragePooling2D1 (Glob                                     
 alAveragePooling2D)                                             
                                                                 
Total params: 410,208
Trainable params: 396,128
Non-trainable params: 14,080
_________________________________________________________________


In [6]:
# Get the description of the first layer
# It is important to review the input layer to verify the size of the image (input size)
model.get_layer(name='sequential_1').get_layer(name='model1').summary()

Model: "model1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1_pad (ZeroPadding2D)      (None, 225, 225, 3)  0           ['input_1[0][0]']                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 16  432         ['Conv1_pad[0][0]']              
                                )                                                                 
                                                                                             

In [9]:
# Output layer summary
# The last layer tells us how many classes are being handled
model.get_layer(name='sequential_3').summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_Dense1 (Dense)        (None, 100)               128100    
                                                                 
 dense_Dense2 (Dense)        (None, 4)                 400       
                                                                 
Total params: 128,500
Trainable params: 128,500
Non-trainable params: 0
_________________________________________________________________


##Evaluar modelo

In [11]:
# Load the labels
# Separate with newline to make a list of values
with open('labels.txt', 'r') as f:
  labels = f.read().split('\n')

In [12]:
def classify_image(img):
# Resize the image
  image=cv2.resize(img,(224,224))
# Normalize image
  normalized_medical_image = (img.astype(np.float32) / 127.0) -1
# Generate an array to save images and test the model
  data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
# Load an image into the array
  data[0] = normalized_medical_image
# Test model
  prediction=model.predict(data)[0].flatten() # Flatten
  confidences={labels[i]:float(prediction[i]) for i in range (4)}
  return confidences

##Use Gradio

In [13]:
image=gr.inputs.Image(shape=(224,224))
label=gr.outputs.Label(num_top_classes=2)

/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:256: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/outputs.py:196: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


In [14]:
gr.Interface(fn=classify_image,inputs=image,
             outputs=label,capture_session=True).launch()

/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `capture_session` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>